In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
in_path = '/mnt/stsi/stsi1/ptseng/UKBB_Resources/phenos/pejman_request.txt'
raw_out_path = '../regenie_input/PCa_pheno_raw.txt'
out_path = '../regenie_input/PCa_pheno.txt'

In [3]:
matching = pd.read_csv('/mnt/stsi/stsi1/ptseng/UKBB_Resources/phenos/matched_torkamani_pejman_ids.txt',sep='\t',low_memory=False)
matching = matching.rename(columns={
    'Pejman':'pejID',
    'Torkamani':'torkID'
})
matching = matching[['pejID','torkID']]
matching = matching.loc[matching['pejID'] != 'f.eid']
matching = matching.astype('int64')
matching.head()

,pejID,torkID
0,3820588,3760523
1,2456041,1049680
2,3608945,4929987
3,4830048,2158683
4,1067801,5304020


In [4]:
data = pd.read_csv(raw_out_path, sep=' ', low_memory=False)
data.head()

,FID,IID,f.40006.0.0,f.40006.1.0,f.40006.2.0,f.40006.3.0,f.40006.4.0,f.40006.5.0,f.40006.6.0,f.40006.7.0,f.40006.8.0,f.40006.9.0,f.40006.10.0,f.40006.11.0,f.40006.12.0,f.40006.13.0,f.40006.14.0,f.40006.15.0,f.40006.16.0
0,1000011,1000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000026,1000026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000032,1000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000044,1000044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000058,1000058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data = data.rename(columns={
    'IID':'pejID'
})
data['pejID'] = data['pejID'].astype('int64')
data = data.drop('FID',1)
data = data.merge(matching)
data[['FID']] = data['torkID']
data = data.rename(columns={'torkID':'IID'})
data = data[([
    'FID','IID',
    "f.40006.0.0", "f.40006.1.0", "f.40006.2.0", "f.40006.3.0", "f.40006.4.0", "f.40006.5.0", 
    "f.40006.6.0", "f.40006.7.0", "f.40006.8.0", "f.40006.9.0", "f.40006.10.0", "f.40006.11.0", 
    "f.40006.12.0", "f.40006.13.0", "f.40006.14.0", "f.40006.15.0", "f.40006.16.0"
])]
data.head()

,FID,IID,f.40006.0.0,f.40006.1.0,f.40006.2.0,f.40006.3.0,f.40006.4.0,f.40006.5.0,f.40006.6.0,f.40006.7.0,f.40006.8.0,f.40006.9.0,f.40006.10.0,f.40006.11.0,f.40006.12.0,f.40006.13.0,f.40006.14.0,f.40006.15.0,f.40006.16.0
0,3267059,3267059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4406597,4406597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2438420,2438420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5753871,5753871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3391516,3391516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data['Ca'] = False
data['PCa'] = False
for (column_name,column_data) in data.iteritems():
    if column_name == 'FID' or column_name == 'IID' or column_name == 'PCa' or column_name == 'Ca':
        continue
    else:
        data['PCa'] |= (column_data == 'C61')
        data['Ca'] |= column_data.notna()
        
for (column_name,column_data) in data.iteritems():
    if column_name == 'FID' or column_name == 'IID' or column_name == 'PCa' or column_name == 'Ca':
        continue
    else:
        data['PCa'] &= ((column_data == 'C61') | column_data.isna())

print(sum(list(data['PCa'])))
print(sum(list(data['Ca'])))

7624
84512


In [7]:
subdata = data.loc[:,['FID','IID','PCa']]
subdata.loc[:,'PCa'] = subdata.loc[:,'PCa'].astype('int8')
subdata.to_csv(out_path,sep = ' ',index = False,)

In [8]:
len(subdata)

501141

In [9]:
subdata.loc[subdata["PCa"]==1,"IID"].to_csv("../../PCa_IDs",index=False)

In [14]:
print(f"{round(len(subdata.loc[subdata['PCa']==1])/len(subdata)*100,2)}%")

1.52%


In [11]:
subdata.loc[subdata["IID"].duplicated()]

,FID,IID,PCa


In [12]:
Ca_subset = data.loc[data['Ca']]

nonPCa_Ca_subset = Ca_subset.loc[(not item for item in Ca_subset['PCa'])]

len(nonPCa_Ca_subset)

nonPCa_Ca_subset[['FID','IID']].to_csv('../regenie_input/exclude_nonPCa_Ca.txt',header=False,index=False,sep=' ')